In [ ]:
name_PDF = input('Введите имя PDF ')
name_PDF = ''+name_PDF+'.pdf'

import fitz
import re
from PyPDF2 import PdfWriter, PdfReader
import os
from PIL import Image
import cv2
import fitz
import io
import sys
import time
from os import walk
from fastai.vision.all import *
import numpy as np


# Получение количества страниц в pdf
reader = PdfReader(name_PDF)
pdf_page_count = len(reader.pages)

In [ ]:
# Извлечение из pdf страниц в формате png
pdffile = name_PDF
doc = fitz.open(pdffile)

if not os.path.exists("delite_str/"):
    os.mkdir('delite_str')

for i in range(pdf_page_count):
    page = doc.load_page(i)  # number of page
    pix = page.get_pixmap()
    pix.save("delite_str/"+str(i)+".png")

In [ ]:
# Удаление 1 и 3 страницы в pdf
from os import walk
l2 = next(walk('delite_str'), (None, None, []))[2]
for i in range(len(l2)):
    if l2[i] == '0.png' or l2[i] == '2.png':
        directory = (''+'delite_str')
        os.remove(f'{directory}//{l2[i]}')

In [ ]:
# Нейросеть удаляет страницы только с текстом
from os import walk
l3 = next(walk(''+'delite_str'), (None, None, []))[2]

learn = load_learner(r'model_+_or_-94.pkl')

for i in range(len(l3)):
    element = learn.predict(item=r'delite_str/'+str(l3[i]))
    element = str(element)
    element = re.search(r"\'([+-_]+)\'", element)
    element = element.group(1)
    
    if element == '-':
        directory = (''+'delite_str')
        os.remove(f'{directory}//{l3[i]}')

In [ ]:
# Получаем массив номеров страниц, которые нужно сохранить и сохраняем измененный pdf
from os import walk
l4 = next(walk(''+'delite_str'), (None, None, []))[2]
for i in range(len(l4)):
    l4[i]= int(l4[i].rstrip('.png'))
l4.sort()
        
pages_to_keep = l4
infile = PdfReader(''+name_PDF+'', 'rb')
output = PdfWriter()

for i in pages_to_keep:
    p = infile.pages[i] 
    output.add_page(p)

with open(''+name_PDF+'','wb') as f:
    output.write(f)
    
shutil.rmtree('delite_str')

In [ ]:
# Извлечение из pdf по слоям страниц
dimlimit = 100
imgdir = 'images'
if not os.path.exists(imgdir):
    os.mkdir(imgdir)

number_pdf = []

pdf_file = fitz.open(r""+name_PDF+"")

location = (r""+imgdir+"")

number_of_pages = len(pdf_file)
for current_page_index in range(number_of_pages):
  print(current_page_index)
  for img_index,img in enumerate(pdf_file.get_page_images(current_page_index)):
        xref = img[0]
        width = img[2]
        height = img[3]
        if min(width, height) <= dimlimit:
            continue
        image = fitz.Pixmap(pdf_file, xref)

        if image.n < 5:        
            image.save("{}/image{}-{}.png".format(location,current_page_index, img_index))
        else:                
            new_image = fitz.Pixmap(fitz.csRGB, image)
            new_image.save("{}/image{}-{}.png".foramt(location,current_page_index, img_index))

In [ ]:
# Удаление картинок-сканов
a_del = input('Введите число ')

from os import walk
filenames = next(walk(''+'images'), (None, None, []))[2]
directory = (''+'images')
for i in range(len(filenames)):
  if (filenames[i].join(filenames[i].split('-')[-1:])) == ''+str(a_del)+'.png':
    os.remove(f'{directory}//{filenames[i]}')

In [ ]:
# Дробление картинок на более мелкие части. Процесс перевода их в серые тона и поиск границ
chet = 0

filenames = next(walk('images'), (None, None, []))[2]
l = len(filenames)
for i in range(l):
  image = cv2.imread('images\\'+filenames[i]+'')
  original = image.copy()
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #blurred = cv2.GaussianBlur(gray, (3, 3), 0)
  thresh = cv2.threshold(gray, 230,255,cv2.THRESH_BINARY_INV)[1]

  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]

  image_number = 0
  for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
      ROI = original[y:y+h, x:x+w]
      cv2.imwrite('images\\'+"ROI_{}"+str(chet)+".png".format(image_number), ROI)
      image_number += 1
      chet = chet+1

  cv2.waitKey(0)

l = next(walk('images'), (None, None, []))[2]
print(len(l))

In [ ]:
# Удаление картинок, из которых были получены дробленные картинки
a_del = input('Введите число ')

from os import walk
filenames = next(walk(''+'images'), (None, None, []))[2]
directory = (''+'images')
for i in range(len(filenames)):
  if (filenames[i].join(filenames[i].split('-')[-1:])) == ''+str(a_del)+'.png':
    os.remove(f'{directory}//{filenames[i]}')

In [ ]:
# Удаление картинок маленького размера
directory = 'images'
files = os.listdir(directory)

for i in files:
    if i.endswith('.png'):
        path = f'{directory}\\{i}'
        im = Image.open(path)
        width, height = im.size
        im.close()
        if width <= 150 or height <= 150:
            try:
                print(path)
                os.remove(path)
            except:
                print('Error')

l = next(walk('images'), (None, None, []))[2]
print(len(l))

In [ ]:
# Нейросеть удаляет картинки где только текст
from os import walk
l2 = next(walk('images'), (None, None, []))[2]

from fastai.vision.all import *
learn = load_learner(r'model_illistration_or_texyonly98.pkl')

for i in range(len(l2)):
    element = learn.predict(item=r'images/'+str(l2[i]))
    element = str(element)
    element = re.search(r"\'([A-Za-z0-9-_]+)\'", element)
    element = element.group(1)
    if element == 'text-only':
        directory = 'images'
        os.remove(f'{directory}\\{l2[i]}')